In [6]:
import numpy as np
import glob, json
import argparse
import os, time, tqdm, datetime
import subprocess

parser = argparse.ArgumentParser()
parser.add_argument('--vid', type=int, nargs='+', default=[])
parser.add_argument('--local', action='store_true', default=False)
parser.add_argument('--mount_dir', required=True)
parser.add_argument('--dataset_dir', required=True)
parser.add_argument('--curr_vid', type=int, default=9)
parser.add_argument('--gen_missing_sj', action='store_true', default=False)

args = argparse.Namespace(
    vid = [8, 10],
    curr_vid = 9,
    mount_dir = '/data/mint/dataset_generation_symlink/',
    dataset_dir = '/data/mint/dataset_generation/',
    gen_missing_sj = False,
)

# Generate dataset path
name = 'random_target'
model_name = "/log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml_step=250/"
misc = "/ema_085000/train/render_face/reverse_sampling"
  
t_start = time.time()
sj_dict_v = {}
sj_dict_all = {}
for id in args.vid + [args.curr_vid]:
    sj_dict_v[id] = {}
    sj_folder = glob.iglob(f'{args.mount_dir}/mount/v{id:02d}/{name}/{model_name}/{misc}/*')
    for sj in sj_folder:
        sj_name = sj.split('/')[-1]
        target_name = glob.iglob(f'{args.mount_dir}/mount/v{id:02d}/{name}/{model_name}/{misc}/{sj_name}/*')
        sj_dict_v[id][sj_name] = 1
        if sj_name not in sj_dict_all.keys():
            sj_dict_all[sj_name] = {'count': 1}
        else: 
            sj_dict_all[sj_name]['count'] += 1
    print(f"[#] V{id:02d} has {len(sj_dict_v[id])} sj...")
        
print(f"[#] Total time: {time.time() - t_start}")
print(f"[#] Total sj: {len(sj_dict_all.keys())}")

template_sj = dict.fromkeys([f'src={i}.jpg' for i in range(0, 60000)])
unvisited_sj = set(template_sj.keys()) - set(sj_dict_all.keys())
print(f"[#] Unvisited sj: {unvisited_sj}")
print(f"[#] Unvisited sj: {len(unvisited_sj)}")

if args.gen_missing_sj:
    # Find index of unvisited sj in /home/mint/Dev/DiFaReli/difareli-faster/dataset_generation/sampler/generated_dataset_seed=47.json
    with open('/home/mint/Dev/DiFaReli/difareli-faster/dataset_generation/sampler/generated_dataset_seed=47.json', 'r') as f:
        dat = json.load(f)
        
    # Find pair-id based on unvisited_sj
    found_pairs = []
    found_id = []
    found_pairs_json = {'pair': {}}
    
    for sj in tqdm.tqdm(list(unvisited_sj)):
        for pair_id, pair_data in dat['pair'].items():
            if f'src={pair_data["src"]}' == sj:
                found_pairs.append(pair_id)
                found_id.append(pair_data["src"].split('=')[-1].split('.')[0])
                found_pairs_json['pair'][pair_id] = pair_data
                break

    # Print or use the found pair-ids as needed
    print("[#] Found pair-ids:", found_pairs)
    print("[#] Found pair-ids:", len(found_pairs))
    found_pairs_json['time'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(f'/home/mint/Dev/DiFaReli/difareli-faster/sample_scripts/dataset_generation/sample_json/fill_sj_generated_dataset_seed=47.json', 'w') as f:
        json.dump(found_pairs_json, f, indent=4)
        
# Just get the first pair of each sj to fulfill the coverage

[#] V08 has 0 sj...
[#] V10 has 0 sj...
[#] V09 has 0 sj...
[#] Total time: 0.0014939308166503906
[#] Total sj: 0
[#] Unvisited sj: {'src=3112.jpg', 'src=48382.jpg', 'src=34602.jpg', 'src=55141.jpg', 'src=20302.jpg', 'src=19390.jpg', 'src=58729.jpg', 'src=14793.jpg', 'src=17631.jpg', 'src=29308.jpg', 'src=2683.jpg', 'src=57405.jpg', 'src=37557.jpg', 'src=54946.jpg', 'src=57198.jpg', 'src=38445.jpg', 'src=35487.jpg', 'src=33883.jpg', 'src=12872.jpg', 'src=12032.jpg', 'src=5568.jpg', 'src=23920.jpg', 'src=9789.jpg', 'src=34900.jpg', 'src=42828.jpg', 'src=14800.jpg', 'src=16766.jpg', 'src=5966.jpg', 'src=2385.jpg', 'src=53062.jpg', 'src=55083.jpg', 'src=37518.jpg', 'src=59318.jpg', 'src=38594.jpg', 'src=16199.jpg', 'src=43252.jpg', 'src=56797.jpg', 'src=33482.jpg', 'src=53671.jpg', 'src=21659.jpg', 'src=55093.jpg', 'src=26648.jpg', 'src=18740.jpg', 'src=16831.jpg', 'src=30501.jpg', 'src=27188.jpg', 'src=16351.jpg', 'src=59379.jpg', 'src=32881.jpg', 'src=782.jpg', 'src=9375.jpg', 'src=1084

# Examine sampling coverage

In [ ]:
print("="*50)
print("Source occurence")
print("Min-Max occurence of source image : ", np.min(source_occur), np.max(source_occur))
print("Mean occurence of source image : ", np.mean(source_occur))
print("Std occurence of source image : ", np.std(source_occur))
print("Unique occurence of source image : ", np.unique(source_occur))
print("="*50)

print("Target occurence")
print("Min-Max occurence of target image : ", np.min(target_occur), np.max(target_occur))
print("Mean occurence of target image : ", np.mean(target_occur))
print("Std occurence of target image : ", np.std(target_occur))
print("Unique occurence of target image : ", np.unique(target_occur))
print(f"Number of occurence <>? : ", len(np.where(target_occur == 0)[0]))
print("="*50)

In [ ]:
# Plot histogram of occurence
import plotly.graph_objects as go


# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(len(source_occur))), y=source_occur,
                    mode='lines+markers',
                    name='Source'))
fig.add_trace(go.Scatter(x=list(range(len(target_occur))), y=target_occur,
                    mode='lines+markers',
                    name='Target'))

fig.show()


# Try load file

In [2]:
import time, json
f = open('./generated_dataset_seed=47.json', 'r')
timing = time.time()
sample_pairs = json.load(f)['pair']
print(f"Time to load json : {time.time() - timing}")

Time to load json : 0.7823045253753662
